In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

MODEL_NUMBER = 2

    
# seed so random stuff isnt random
SEED = 100

# transforms
transform = transforms.Compose([

    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

dataset = torchvision.datasets.EuroSAT(root='./data', download=True, transform=transform)

# split into train, test
# 80% train, 20% test 
train_ratio = 0.8
test_ratio = 0.2


total_size = len(dataset)
train_size = int(total_size * train_ratio)
test_size = total_size - train_size


train_dataset, test_dataset = random_split(
    dataset, [train_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"train: {len(train_dataset)}")
print(f"test size: {len(test_dataset)}")

train: 21600
test size: 5400


In [8]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.relu = torch.nn.ReLU()

        self.c1 = torch.nn.Conv2d(3, 8, 3, padding=1)
        self.c2 = torch.nn.Conv2d(8, 32, 3, padding=1)
        self.maxpool1 = torch.nn.MaxPool2d(2)
        self.dropout1 = torch.nn.Dropout2d(0.25)
        self.c3 = torch.nn.Conv2d(32, 128, 3, padding=1)
        self.maxpool2 = torch.nn.MaxPool2d(2)
        self.dropout2 = torch.nn.Dropout2d(0.25)
        self.c5 = torch.nn.Conv2d(128, 128, 3, padding=1)
        self.maxpool3 = torch.nn.MaxPool2d(2)
        self.dropout3 = torch.nn.Dropout2d(0.25)
        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(128 * 8 * 8, 256)
        self.dropout4 = torch.nn.Dropout(0.5)
        self.fc2 = torch.nn.Linear(256, 10)

    def forward(self, x):
        x = self.c1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.dropout1(x)
        x = self.c3(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.dropout2(x)
        x = self.c5(x)
        x = self.relu(x)
        x = self.maxpool3(x)
        x = self.dropout3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

model = Model()
print(f"Model: {model}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")



Model: Model(
  (relu): ReLU()
  (c1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c2): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (c3): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout2d(p=0.25, inplace=False)
  (c5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout3): Dropout2d(p=0.25, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (dropout4): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Total parameters: 2287114


In [9]:

# dataloaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=128, 
    shuffle=True,
    pin_memory=True, # for use on gpu
    num_workers=4 # arbitrarily chosen. some sources recommend 4 per gpu
)


test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    pin_memory=True,
    num_workers=4
)

In [10]:

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    model.to(device, non_blocking=True) # according to pytorch docs we can do this if memory is pinned
else:  
    model.to(device)
    


In [11]:
def train_one_epoch():
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def test_one_epoch():
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [12]:
# tensorboard setup
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/train_{}_{}'.format(MODEL_NUMBER, timestamp))

checkpoint_dir = f'checkpoints/train_{MODEL_NUMBER}_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)

EPOCHS = 200

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    


    train_loss, train_acc = train_one_epoch()
    
    test_loss, test_acc = test_one_epoch()
    
    print(f'train loss: {train_loss:.4f} | test acc: {train_acc:.2f}%')
    print(f'test loss:  {test_loss:.4f} | test acc:  {test_acc:.2f}%')
    
    # log data
    writer.add_scalar('loss/1_train', train_loss, epoch + 1)
    writer.add_scalar('loss/2_test', test_loss, epoch + 1)
    writer.add_scalar('accuracy/1_train', train_acc, epoch + 1)
    writer.add_scalar('accuracy/2_test', test_acc, epoch + 1)
    writer.flush()
    
    # checkpoint every 20 epochs
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f'checkpoint saved: {checkpoint_path}')
    
    print()

# save final model
final_checkpoint_path = os.path.join(checkpoint_dir, f'model_final.pt')
torch.save(model.state_dict(), final_checkpoint_path)
print(f'final checkpoint saved: {final_checkpoint_path}')

writer.close()
print('Complete')

Epoch 1/200
train loss: 1.8201 | test acc: 30.12%
test loss:  1.4734 | test acc:  43.59%

Epoch 2/200
train loss: 1.3959 | test acc: 46.93%
test loss:  1.1712 | test acc:  55.28%

Epoch 3/200
train loss: 1.2214 | test acc: 54.06%
test loss:  1.0227 | test acc:  62.87%

Epoch 4/200
train loss: 1.1121 | test acc: 59.55%
test loss:  0.9298 | test acc:  66.57%

Epoch 5/200
train loss: 1.0245 | test acc: 63.25%
test loss:  0.8288 | test acc:  69.93%

Epoch 6/200
train loss: 0.9805 | test acc: 65.07%
test loss:  0.8685 | test acc:  68.13%

Epoch 7/200
train loss: 0.9258 | test acc: 66.97%
test loss:  0.7694 | test acc:  72.26%

Epoch 8/200
train loss: 0.8867 | test acc: 67.96%
test loss:  0.7405 | test acc:  72.65%

Epoch 9/200
train loss: 0.8578 | test acc: 69.63%
test loss:  0.7315 | test acc:  73.22%

Epoch 10/200
train loss: 0.8304 | test acc: 70.54%
test loss:  0.6769 | test acc:  75.80%

Epoch 11/200
train loss: 0.8113 | test acc: 71.23%
test loss:  0.6924 | test acc:  74.93%

Epoch 12